In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
import pyspark.sql.functions as F


In [0]:

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

### Lendo Csv

In [0]:
path_orders = '/FileStore/transient/olist/orders'
df_orders = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\"")\
.option("inferSchema",True)\
.load(path_orders)
#transient\csv\olist

In [0]:
path_payments = '/FileStore/transient/olist/payments'
df_payments = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\"")\
.option("inferSchema",True)\
.load(path_payments)

In [0]:
path_employees = '/FileStore/transient/departments/employees'
df_employees = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\'")\
.option("inferSchema",True)\
.load(path_employees)

### Selecionando e manipulando os dados

In [0]:
df_orders.select('order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',\
                'order_delivered_customer_date','order_estimated_delivery_date').display()

In [0]:
df_orders.printSchema()

In [0]:
df_orders.select('order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',\
                'order_delivered_customer_date','order_estimated_delivery_date').display()

### Colunas

As colunas são unidades de manipulação de dados do Spark. 
Podemos referencias colunas de algumas formas <br>
* col('nome_coluna') <br>
* dataframe['nome_coluna'] <br>
* dataframe.nome_coluna <br>

In [0]:
from pyspark.sql.functions import col, round
(
df_orders.select('order_id', 'customer_id', 'order_status', 
F.split(F.col('order_approved_at'), '-').getItem(0).alias('aproved_year_at'),
F.split(df_orders['order_approved_at'], '-').getItem(1).alias('aproved_month_at'),
F.split(df_orders.order_approved_at, '-').getItem(2).alias('aproved_day_at')).display()
)

### Criando novas colunas

In [0]:
from pyspark.sql.functions import col, round

df_orders.withColumn('aproved_year_at', F.split(F.col('order_approved_at'), '-').getItem(0))\
.withColumn('aproved_month_at',F.split(df_orders['order_approved_at'], '-').getItem(1))\
.withColumn('aproved_day_at',F.split(F.split(df_orders.order_approved_at, '-').getItem(2),' ').getItem(0)  )\
.withColumn('country',F.lit('BR') ).display()


### Renomeando colunas

In [0]:

df_orders_renamed = df_orders.withColumnRenamed("order_id","id_pedido") \
    .withColumnRenamed("customer_id","id_cliente") \
    .withColumnRenamed("order_status","status_pedido") \
    .withColumnRenamed("order_purchase_timestamp","pedido_data_hora") \
    .withColumnRenamed("order_approved_at","aprovado_em") \
    .withColumnRenamed("order_delivered_carrier_date","data_entrega") \
    .withColumnRenamed("order_delivered_customer_date","data_entrega_cliente") \
    .withColumnRenamed("order_estimated_delivery_date","_data_entrega_estimada") 


In [0]:
df_orders_renamed.display()

### Expressões

In [0]:
from pyspark.sql.functions import expr

(
    df_orders.select('order_id', 'customer_id', 'order_status', 
    F.expr('upper(order_status)'), )                    
    .display()
)

In [0]:
from pyspark.sql.functions import expr
(
    df_orders.select('order_id', 'customer_id', 'order_status', 
    expr('upper(order_status)'),
    expr('substring(order_approved_at, 0,4) as year'),
    expr('substring(order_approved_at, 6,2) as month'),
    expr('substring(order_approved_at, 9,2) as day'))                    
    .display()
)

In [0]:

cols = ['order_id', 'order_status', 'order_estimated_delivery_date']
df_orders.select(cols).display()

In [0]:
cols = ['order_id', 'order_status', 'order_estimated_delivery_date']
df_orders.select('customer_id', *cols).display()

Observações:
* Podemos realizar operações sobre colunas selecionadas. 
* O DataFrame resultante resultante das operações vai obedeçer a order das colunas em que ele foi criado.

In [0]:
df_orders.display()

In [0]:
df_orders_selected =  (
    df_orders.select('order_id', 'customer_id', 'order_status', 
    expr('upper(order_status)'),
    expr('substring(order_approved_at, 0,4) as year'),
    expr('substring(order_approved_at, 6,2) as month'),
    expr('substring(order_approved_at, 9,2) as day'))   
)

In [0]:
df_orders_selected.selectExpr('order_id', 'customer_id','upper(order_status) as order_status','concat(year,"-",month,"-",day) as date').display()

###  Selecionando valores únicos

In [0]:
df_orders_selected.select('year').distinct().display()

In [0]:
df_orders_selected.dropDuplicates(subset=['year']).display()

### Filtrando registros e condições

Operadores lógicos disponíveis:
* e: &
* ou: |
* não: ~

As funções `filter()` e `where()` podem ser utilizadas no processo de filtragem.

In [0]:
df_orders_selected.filter(~(col('year') == 'null')).display()

In [0]:
df_orders_selected.filter((col('year').isNull())).display(5)

In [0]:
df_orders_selected.select('order_status').distinct().display()

In [0]:
(
    df_orders_selected.filter((col('year') == '2016') & (col('order_status') == 'invoiced'))
    .display()
)

In [0]:
(
    df_orders_selected.filter(((col('order_status') == 'unavailable') | (col('order_status') == 'canceled')) & (col('year') == '2017')).display()
  
)

In [0]:
(
    df_orders_selected.filter((col('order_status').isin('unavailable', 'canceled')) & (col('year') == '2017')).display()
)

In [0]:

(
    df_orders_selected
    .filter((col('order_status').isin('unavailable', 'canceled')))
    .filter((col('year') == '2017'))
    .display()
)

### Utilizando expressões no filtro

In [0]:
(
    df_orders_selected
    .filter('order_status in ("unavailable", "canceled") and year == "2017"')
    .display()
)

#### Observações
Quando nos referimos às colunas por meio da função `col()`, temos acesso à diversos métodos das colunas que podem ser utilizados para auxliar na filtragem do DataFrame. Alguns deles são:
* `isin()`: checa se a coluna contém os valores listados na função.
* `contains()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Aceita uma outra coluna de texto.
* `like()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Funciona de forma similar ao "LIKE" do SQL.
* `rlike()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (**aceita regex**). Funciona de forma similar ao "RLIKE" do SQL.
* `startswith()`: utilizado para verificar se uma coluna de texto começa com algum padrão especificado (**aceita regex**).
* `endswith()`: utilizado para verificar se uma coluna de texto termina com algum padrão especificado (**aceita regex**).
* `between()`: checa se os valores da coluna estão dentro do intervalo especificado. Os dois lados do intervalo são inclusivos.
* `isNull()`: retorna True se o valor da coluna é nulo
* `isNotNull()`: retorna True se o valor da coluna não é nulo

Outros métodos úteis:
* `alias()/name()`: usado para renomear as colunas em operações como select() e agg()
* `astype()/cast()`: usado para mudar o tipo das colunas. Aceita tanto um string como um tipo especificado pelo módulo pyspark.sql.types
* `substr()`: utilizado para cortar um string com base em índices dos caracteres 

#### Funções numéricas
* `round()`: arredonda o valor numérico
* `ceil()`: arredonda o valor numérico para o maior inteiro mais próximo
* `floor()`: arredonda o valor numérico para o menor inteiro mais próximo
* `sqrt()`: retorna a raiz quadrada do valor
* `exp()`: retorna a exponencial do valor
* `log()`: retorna a logaritmo natural do valor
* `log10()`: retorna a logaritmo na base 10 do valor
* `greatest()`: retorna o maior valor dentre os valores das colunas. Análogo ao `max()`, mas entre colunas
* `least()`: retorna o menor valor dentre os valores das colunas. Análogo ao `min()`, mas entre colunas

In [0]:
df_payments.display()

In [0]:
df_payments.select(F.round('payment_value',1)).display()

In [0]:
df_payments.select('*',F.floor('payment_value').alias('payment_value_floor')).display()

In [0]:
df_payments.select('*',F.ceil('payment_value').alias('payment_value_ceil')).display()

#### Funções para Texto
* `upper()`: retorna o string em letras maiúsculas
* `lower()`: retorna o string em letras minúsculas
* `initcap()`: retorna a primeira letra de cada palavra no string em letras maiúsculas
* `trim()`: retira os espaços em branco do início e do final do string
* `ltrim() / rtrim()`: retira os espaços em branco do início e do final do string, respectivamente
* `lpad() / rpad()`: acrescenta um caractere no início e no final do string, respectivamente, até que o string tenha um determinado comprimento
* `length()`: retorna o comprimento do string, em quantidade de caracteres
* `split()`: quebra o string a partir de um padrão e retorna um array com os string resultantes
* `concat()`: concatena uma ou mais colunas de string
* `concat_ws()`: concatena uma ou mais colunas de string, com um separador entre elas
* `regexp_extract()`: retorna um match no string a partir de um padrão regex
* `regexp_replace()`: substitui um mtach no strinf a partir de um padrão regex com outros caracteres
* `substring()`: retorna os caracteres do string que estão entre dos indices especificados. Análogo a `f.col().substring()`

In [0]:
df_employees.display()

In [0]:
df_employees.select(F.split('email','@')).display()

In [0]:
(df_employees.select('*',
                    F.concat('first_name',F.lit(' '),'last_name').alias('full_name'),
                    F.split('email','@').getItem(1).alias('email_company'),
                    F.upper('email').alias('email_upper'),
                    F.length('email').alias('len_email'),
                    F.length(F.trim('email')).alias('len_email_trim')
                    ).display()
)
                    

#### Funções para data
* `add_months()`: retorna a data depois de adicionar "x" meses
* `months_between()`: retorna a diferença entre duas datas em meses
* `date_add()`: retorna a data depois de adicionar "x" dias
* `date_sub()`: retorna a data depois de subtrair "x" dias
* `next_day()`: retorna o dia seguinte de alguma data
* `datediff()`: retorna a diferença entre duas datas em dias
* `current_date()`: retorna a data atual
* `dayofweek() / dayofmonth() / dayofyear()`: retorna o dia relativo à semana, ao mês e ao ano, respectivamente
* `weekofyear()`: retorna a semana relativa ao ano
* `second() / minute() / hour()`: retorna os segundos, os minutos e as horas de uma coluna de date-time, respectivamente
* `month() / year()`: retorna o mês e o ano de uma coluna de data, respectivamente
* `last_day()`: retorna o último dia do mês do qual a data considerada pertence
* `to_date()`: transforma a coluna no tipo data (t.DateType())
* `trunc()`: formata a data para a unidade especificada
    * `year`: "{ano}-01-01"
    * `month`: "{ano}-{mes}-01"

In [0]:
df_orders.display()

In [0]:
(df_orders.select('order_purchase_timestamp',
                  'order_delivered_customer_date',
                  F.year('order_purchase_timestamp'),
                  F.add_months('order_purchase_timestamp',3),
                  F.datediff('order_delivered_customer_date','order_purchase_timestamp'))
 .display()
 )


#### Funções para Arrays
* `array()`: constrói um array com as colunas selecionadas
* `flatten()`: transforma um array de arrays em um unico array
* `explode()`: retorna uma nova linha para cada elemento no array 
* `size()`: retorna o número de elementos no array
* `sort_array()`: ordena os elementos do array, de forma crescente ou decrescente
* `reverse()`: reverte a ordem dos elementos de um array
* `array_distinct()`: remove elementos duplicados do array
* `array_contains()`: verifica se o array contém o elemento especificado
* `arrays_overlap()`: partir de 2 colunas de arrays, verifica se elas tem algum elemento em comum, retornando True ou False
* `array_union()`: a partir de 2 colunas de arrays, retorna um array com os elementos unidos das duas colunas, sem duplicatas
* `array_except()`: a partir de 2 colunas de arrays, retorna um array com os elementos que estão em uma coluna mas não estão na outra, sem duplicatas
* `array_intersect()`: a partir de 2 colunas de arrays, retorna um array com os elementos que nas duas colunas, sem duplicatas
* `array_join()`: retorna um string após concatenar os elementos do array usando o delimitador especificado
* `array_max() / array_min()`: retorna o máximo e o mínimo valor do array, respectivamente
* `array_remove()`: remove todos os elementos do array que são iguais ao valor especificado

In [0]:
data = [
 ("João,,Silva",["Excel","VBA","Contatabilidade"],["Contador","Financeiro"],"MG","SP"),
 ("Rosália,lima,",["Costura","Cozinha","Educação"],["Dona de Casa","Avó"],"PI","SP"),
 ("Severino,,Matos",["CSharp","VB"],["Desenvolvedor","Arquiteto"],"CE","SP")
]

from pyspark.sql.types import StringType, ArrayType,StructType,StructField
schema = StructType([ 
    StructField("Nome",StringType(),True), 
    StructField("Conhecimentos",ArrayType(StringType()),True), 
    StructField("Ocupacao",ArrayType(StringType()),True), 
    StructField("EstadoOrigem", StringType(), True), 
    StructField("EstadoAtual", StringType(), True)
  ])

df = spark.createDataFrame(data=data,schema=schema)

In [0]:
from pyspark.sql.functions import explode
df.select(df.Nome,explode(df.Conhecimentos)).display()

* `drop()`: retira do DataFrame as linhas com nulos, com base no que foi passado para o argumento how
    * any (default): retira todas as linhas com pelo menos um valor nulo nas colunas
    * all: somente retira as linhas com todos os valores nulos nas colunas
* `fill()`: preenche os valores nulos no DataFrame com uma constante, passada pelo usuário
* `replace()`: substitui o valor (não somente os valores nulos) por algum outro passado pelo usuário


In [0]:
from pyspark.sql.functions import split
df.select(split(col('Nome'),",").alias("ArrayNome")).display()

In [0]:
from pyspark.sql.functions import array
df.select(df.Nome,array(df.EstadoOrigem,df.EstadoAtual).alias("Estados")).display()

In [0]:
from pyspark.sql.functions import array_contains
df.select(df.Nome,array_contains(df.Conhecimentos,"Cozinha")
    .alias("array_contains")).display()

#### Funções para valores nulos
* `drop()`: retira do DataFrame as linhas com nulos, com base no que foi passado para o argumento how
    * any (default): retira todas as linhas com pelo menos um valor nulo nas colunas
    * all: somente retira as linhas com todos os valores nulos nas colunas
* `fillna()`: preenche os valores nulos no DataFrame com uma constante, passada pelo usuário
* `replace()`: substitui o valor (não somente os valores nulos) por algum outro passado pelo usuário

In [0]:
data = [
    ("João",None,"M"),
    ("Maria","NY","F"),
    ("Valéria",None,None)
  ]

columns = ["nome","estado","sexo"]
df = spark.createDataFrame(data,columns)

In [0]:
df.filter("estado is NULL").display()


In [0]:
df.filter(df.estado.isNull()).display()


In [0]:
df.filter(col("estado").isNull()).display()

In [0]:
df_limpo = df.fillna('indefinido')
df_limpo.display()